<a href="https://colab.research.google.com/github/asigalov61/MIDI-TXT-MIDI/blob/master/MIDI_TXT_MIDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIDI-TXT-MIDI
***
## A full-featured MIDI processor and encoder/decoder for NLP based symbolic music generation and AI model creation/training
***
### Based on absolutely amazing repo-code-colab by Soheil: https://github.com/blu-ray/Music-Generation and on fantastic Python library MIDI.py by Peter Billam https://pjb.com.au/

***

# Supported MIDI features (for now):

## EVENTS (in an "opus" structure)

### 1) 'note_off', dtime, channel, note, velocity    

### 2) 'note_on', dtime, channel, note, velocity

### 3) 'key_after_touch', dtime, channel, note, velocity

### 4) 'control_change', dtime, channel, controller(0-127), value(0-127)

### 5) 'patch_change', dtime, channel, patch

### 6) 'channel_after_touch', dtime, channel, velocity

### 7) 'pitch_wheel_change', dtime, channel, pitch_wheel

### 8) 'track_name', dtime, text

### 9) 'instrument_name', dtime, text

### 10) 'end_track', dtime

### 11) 'set_tempo', dtime, tempo

### 12) 'time_signature', dtime, nn, dd, cc, bb

### 13) 'key_signature', dtime, sf, mi

### 14) 'text_event', dtime, text

### 15) 'track_name', dtime, text

### 16) 'lyric', dtime, text
 

# Data Gathering

In [ ]:
#@title Download MIDI.py and some test data
#MIDI Library
!curl -L "https://pjb.com.au/midi/free/MIDI.py" > 'MIDI.py'

## MIDI format Mozart Data
!wget http://www.piano-midi.de/zip/mozart.zip
!sudo apt-get install unzip
!unzip mozart.zip -d Dataset

In [ ]:
#@title Process MIDI to TXT
enable_sampling = False #@param {type:"boolean"}
sample_length_in_notes = 997 #@param {type:"slider", min:0, max:2000, step:1}
allow_tempo_changes = True #@param {type:"boolean"}
parse_text_fields_for_nlp = True #@param {type:"boolean"}

# MIDI Dataset to txt converter 
import MIDI
import os
import numpy as np
import tqdm.auto

def write_notes(file_address):
    midi_file = open(file_address, 'rb')
    #print('Processing File:', file_address)
    score = MIDI.midi2opus(midi_file.read())
    midi_file.close()
    # ['note', start_time, duration, channel, note, velocity]

    itrack = 1


    notes = []

    tokens = []

    this_channel_has_note = False

    file = open('Dataset.txt', 'a')
    file.write('[MIDI-TXT-MIDI Textual Music Dataset] ')
    while itrack < len(score):
        for event in score[itrack]:

            if event[0] == 'note_off':
                this_channel_has_note = True
                notes.append(event[3])

                
                tokens.append([event[3], event[4], event[1]])
                
                file.write('Nf' + ' Du' + str(event[1]) + ' Ch' + str(event[2]) + ' Nt' + str(event[3]) + ' Ve' + str(event[4]) + ' ')


            if event[0] == 'note_on':
                this_channel_has_note = True
                notes.append(event[3])
                
                tokens.append([event[3], event[4], event[1]])

                file.write('No' + ' Du' + str(event[1]) + ' Ch' + str(event[2]) + ' Nt' + str(event[3]) + ' Ve' + str(event[4]) + ' ')

            if event[0] == 'key_after_touch':
                this_channel_has_note = True

                
                tokens.append([event[3], event[4], event[1]])
                file.write('Ka' + ' Du' + str(event[1]) + ' Ch' + str(event[2]) + ' Nt' + str(event[3]) + ' Ve' + str(event[4]) + ' ')

            if event[0] == 'control_change':
                this_channel_has_note = True

                
                tokens.append([event[3], event[4], event[1]])
                
                file.write('Cc' + ' Du' + str(event[1]) + ' Ch' + str(event[2]) + ' Co' + str(event[3]) + ' Cv' + str(event[4]) + ' ')

            if event[0] == 'patch_change':
                this_channel_has_note = True

                
                tokens.append([event[3], event[2], event[1]])
                
                file.write('Pc' + ' Du' + str(event[1]) + ' Ch' + str(event[2]) + ' Pt' + str(event[3]) + ' ')

            if event[0] == 'channel_after_touch':
                this_channel_has_note = True

                
                tokens.append([event[3], event[4], event[1]])
                
                file.write('Ct' + ' Du' + str(event[1]) + ' Ch' + str(event[2]) + ' Ve' + str(event[3]) + ' ')

            if event[0] == 'pitch_wheel_change':
                this_channel_has_note = True

                
                tokens.append([event[3], event[2], event[1]])
                
                file.write('Pc' + ' Du' + str(event[1]) + ' Ch' + str(event[2]) + ' Pw' + str(event[3]) + ' ')

            if event[0] == 'instrument_name':
              if parse_text_fields_for_nlp:
                this_channel_has_note = True

                
                tokens.append([event[3], event[4], event[1]])
                
                file.write('In' + ' Dt' + str(event[1]) + ' Tx' + str(event[2]) + ' ')

            if event[0] == 'end_track':
                this_channel_has_note = True

                
                tokens.append([event[3], event[4], event[1]])
                
                file.write('Et' + ' Dt' + str(event[1]) + ' ')

            if event[0] == 'set_tempo':
              if allow_tempo_changes:
                this_channel_has_note = True

                
                tokens.append([ event[2], event[1]])
                
                file.write('St' + ' Dt' + str(event[1]) + ' Tm' + str(event[2]) + ' ')


            if event[0] == 'time_signature':
                this_channel_has_note = True

                
                tokens.append([event[3], event[4], event[1]])
                
                file.write('Ts' + ' Dt' + str(event[1]) + ' nn' + str(event[2]) + ' dd' + str(event[3]) + ' cc' + str(event[4]) + ' bb' + str(event[5]) +' ')

            if event[0] == 'key_signature':
                this_channel_has_note = True
                
                tokens.append([event[3], event[2], event[1]])
                
                file.write('Ks' + ' Dt' + str(event[1]) + ' sf' + str(event[2]) + ' mi' + str(event[3]) + ' ')

            if event[0] == 'track_name':
              if parse_text_fields_for_nlp:
                this_channel_has_note = True
                
                tokens.append([event[2], event[1]])
                
                file.write('Tn' + ' Dt' + str(event[1]) + ' Tx' + str(event[2]) + ' ')

            if event[0] == 'text_event':
              if parse_text_fields_for_nlp:
                this_channel_has_note = True
                
                tokens.append([ event[2], event[1]])
                
                file.write('Te' + ' Dt' + str(event[1]) + ' Tx' + str(event[2]) + ' ')

            if event[0] == 'lyric':
              if parse_text_fields_for_nlp:
                this_channel_has_note = True
                
                tokens.append([ event[2], event[1]])
                
                file.write('Ly' + ' Dt' + str(event[1]) + ' Tx' + str(event[2]) + ' ')




        itrack += 1
        if this_channel_has_note and len(notes) > sample_length_in_notes:
          if enable_sampling:
            break
    file.close()     
       

dataset_addr = "Dataset"
files = os.listdir(dataset_addr)
for file in tqdm.auto.tqdm(files):
    path = os.path.join(dataset_addr, file)
    write_notes(path)

# Model Training

In [ ]:
#@title Define Constants and Functions
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

BATCH_SIZE = 16
SEQ_LENGTH = 64

def get_data_unique_chars_dicts(data):
    set_of_unique_chars = set(data)
    list_of_unique_chars = sorted(list(set_of_unique_chars))
    char_to_index = {}
    for i in range(len(list_of_unique_chars)):
        char_to_index[list_of_unique_chars[i]] = i
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    return char_to_index, index_to_char

def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE)
    for start in range(0, batch_chars - SEQ_LENGTH, 64):
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))
        for batch_index in range(0, 16):  
            for i in range(0, 64):
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1
        yield X, Y

def make_model(batch_size, seq_length, num_of_unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = num_of_unique_chars, output_dim = 256, 
                        batch_input_shape = (batch_size, seq_length))) 
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(num_of_unique_chars)))
    model.add(Activation("softmax"))
    
    return model

def train(model, model_name, num_of_epochs = 100):
  epoch_report = {}
      
  for epoch in range(num_of_epochs):
      print("Epoch {}/{}".format(epoch+1, num_of_epochs))

      average_epoch_loss = 0
      average_epoch_acc = 0
      counter = 0

      for i, (x, y) in enumerate(read_batches(chars_of_data, num_of_unique_chars)):
          if (i + 1) % 5:
            counter += 1
            batch_loss, batch_accuracy = model.train_on_batch(x, y)
            average_epoch_loss += batch_loss
            average_epoch_acc += batch_accuracy
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, batch_loss, batch_accuracy))
          else:
            batch_loss, batch_accuracy = model.test_on_batch(x, y)
            print(f"TEST Loss: {batch_loss}, Accuracy: {batch_accuracy}")

      epoch_report[epoch] = (average_epoch_loss / (counter), average_epoch_acc / (counter))
      
      if (epoch + 1) % 2 == 0:
          model.save_weights(f"Weights_{model_name}.h5")
          print(f'Saved Weights at epoch {epoch+1} to file Weights_{model_name}.h5')

  for epoch_num in epoch_report.keys():
    epoch_loss, epoch_acc = epoch_report[epoch_num]
    print(f"{epoch_num}\t{epoch_loss}\t{epoch_acc}")

In [ ]:
#@title Train the Model
with open('Dataset.txt', 'r') as data_file:
    data = data_file.read()

char_to_index, index_to_char = get_data_unique_chars_dicts(data)
num_of_unique_chars = len(char_to_index)
chars_of_data = np.asarray([char_to_index[c] for c in data], dtype = np.int32)

model = make_model(BATCH_SIZE, SEQ_LENGTH, num_of_unique_chars)
model.compile(loss = "categorical_crossentropy", optimizer = "Adamax", 
              metrics = ["accuracy"])

train(model, "MIDI-TXT-MIDI", 100)


# Sampling

In [ ]:
#@title Define Functions
import tqdm.auto

def make_model_for_sampling(num_of_unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = num_of_unique_chars, output_dim = 256, 
                        batch_input_shape = (1, 1))) 
  
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, stateful = True)) 
    model.add(Dropout(0.2))
    
    model.add((Dense(num_of_unique_chars)))
    model.add(Activation("softmax"))
    
    return model

def adjust_seq(seq):
  count = 0
  for i in seq:
      count += 1
      if i == "\n":
          break
  beginning_part_removed = seq[count:]

  count = 0
  for i in beginning_part_removed:
      count += 1
      if i == "\n" and beginning_part_removed[count] == "\n":
          break
  ending_part_removed = beginning_part_removed[:count]
  return ending_part_removed

def generate_sequence(model_weights_address, initial_index, 
                      seq_length, is_abc=False):
    sequence_index = [initial_index]

    model = make_model_for_sampling(num_of_unique_chars)
    model.load_weights(model_weights_address)
    
    for i in tqdm.auto.tqdm(range(seq_length)):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        predicted_probs = model.predict_on_batch(batch).ravel()
        sample = np.random.choice(range(num_of_unique_chars), size = 1, p = predicted_probs)
        
        sequence_index.append(sample[0])
    
    seq = ''.join(index_to_char[c] for c in sequence_index)
    
    if is_abc:
      return adjust_seq(seq)
    else:
      return seq

In [ ]:
#@title Generate Output
number_of_tokens_to_generate = 8192 #@param {type:"slider", min:0, max:8192, step:128}
seq = generate_sequence("Weights_MIDI-TXT-MIDI.h5", 0, number_of_tokens_to_generate)
print(seq)
with open('output.txt', 'w') as gen_song_file:
  gen_song_file.write(seq)

# Convert To MIDI

In [ ]:
#@title Convert to MIDI from TXT
number_of_ticks_per_quarter = 425 #@param {type:"slider", min:1, max:1440, step:8}

import MIDI
import tqdm.auto
notes = []
velocities = []
timings = []
durations = []

with open('/content/output.txt', 'r') as file:
    notestring=file.read()

score_note = notestring.split(" ")

score = score_note

i=0

song_score = [number_of_ticks_per_quarter, [['track_name', 0, b'Composed by Artificial Intelligence Model']]]

for i in tqdm.auto.tqdm(range(len(score))):

        # if the event is a blank, space, "eos" or unknown, skip and go to next event
        if score[i] in ["", " ", "<eos>", "<unk>"]:
            continue

        # if the event starts with 'end' indicating an end of note
        elif score[i][:2]=="@@":

            continue

        # in this block, we are looking for notes   
        else:
            # Look ahead to see if an end<noteid> was generated
            # soon after.  


            note_string_len = len(score[i])
            for j in range(1,200):
                if i+j==len(score):
                    break


            if score[i] == 'No':
              try:
                
                song_score[-1].append(['note_on', 
                                       int(score[i+1][2:]), 
                                       int(score[i+2][2:]),
                                       int(score[i+3][2:]),
                                       int(score[i+4][2:])])
                           
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'Nf':
              try:
                song_score[-1].append(['note_off', 
                                       int(score[i+1][2:]), 
                                       int(score[i+2][2:]),
                                       int(score[i+3][2:]),
                                       int(score[i+4][2:])])
                
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'Cc':
              try:
                song_score[-1].append(['control_change',
                                       int(score[i+1][2:]), 
                                       int(score[i+2][2:]),
                                       int(score[i+3][2:]),
                                       int(score[i+4][2:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'Kt':
              try:
                song_score[-1].append(['key_after_touch', 
                                       int(score[i+1][2:]), 
                                       int(score[i+2][2:]),
                                       int(score[i+3][2:]),
                                       int(score[i+4][2:])])
              except:
                print("Unknown note: " + score[i])



            if score[i] == 'Pc':
              try:
                song_score[-1].append(['patch_change', 
                                       int(score[i+1][2:]), 
                                       int(score[i+2][2:]),
                                       int(score[i+3][2:])
                                       ])
              except:
                print("Unknown note: " + score[i])


            if score[i] == 'Ca':
              try:
                song_score[-1].append(['channel_after_touch', 
                                       int(score[i+1][2:]), 
                                       int(score[i+2][2:]),
                                       int(score[i+3][2:])
                                       ])
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'Pw':
              try:
                song_score[-1].append(['pitch_wheel_change', 
                                       int(score[i+1][2:]), 
                                       int(score[i+2][2:]),
                                       int(score[i+3][2:])
                                       ])

              except:
                print("Unknown note: " + score[i])

            if score[i] == 'In':
              try:
                song_score[-1].append(['instrument_name', 
                                       int(score[i+1][2:]), 
                                       int(score[i+2][2:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'Et':
              try:
                song_score[-1].append(['end_track', 
                                       int(score[i+1][2:])])
              except:
                print("Unknown note: " + score[i])

            if score[i] == 'St':
              try:
                song_score[-1].append(['set_tempo', 
                                       int(score[i+1][2:]), 
                                       int(score[i+2][2:])])
              except:
                print("Unknown note: " + score[i])


            if score[i] == 'Ts':
              try:
                song_score[-1].append(['time_signature', 
                                       int(score[i+1][2:]), 
                                       int(score[i+2][2:]),
                                       int(score[i+3][2:]),
                                       int(score[i+4][2:]),
                                       int(score[i+5][2:])])

              except:
                print("Unknown note: " + score[i])


            if score[i] == 'Ks':
              try:
                song_score[-1].append(['key_signature', 
                                       int(score[i+1][2:]), 
                                       int(score[i+2][2:]),
                                       int(score[i+3][2:])])

              except:
                print("Unknown note: " + score[i])



            if score[i] == 'Ly':
              try:
                song_score[-1].append(['lyric', 
                                       int(score[i+1][2:]), 
                                       int(score[i+2][2:])])

              except:
                print("Unknown note: " + score[i])

            if score[i] == 'Te':
              try:
                song_score[-1].append(['text_event', 
                                       int(score[i+1][2:]), 
                                       int(score[i+2][2:])])

              except:
                print("Unknown note: " + score[i])

            if score[i] == 'Tn':
              try:
                song_score[-1].append(['track_name', 
                                       int(score[i+1][2:]), 
                                       int(score[i+2][2:])])

              except:
                print("Unknown note: " + score[i])




midi_data = MIDI.opus2midi(song_score)
with open('output.mid', 'wb') as midi_file:
    midi_file.write(midi_data)

MIDI.score2stats(song_score)